# VIT + Unet

1) load a data for a test
2) update a loop for test
3) for each epoch display a training loss , validation loss and test loss + ssim value also

# Data Loading

## Train

In [1]:
X_train_path = '/home/simlab120/Denoise_comp/Pragyant/image-impeccable-train-data-part1/42487393/seismic_w_noise_vol_42487393.npy'
y_train_path = '/home/simlab120/Denoise_comp/Pragyant/image-impeccable-train-data-part1/42487393/seismicCubes_RFC_fullstack_2024.42487393.npy'

## Test

In [2]:
X_test_path = '/home/simlab120/Denoise_comp/Pragyant/image-impeccable-test-data/2024-06-10_0d6402b1/seismic_w_noise_vol_44319345.npy'
y_test_path = '/home/simlab120/Denoise_comp/Pragyant/image-impeccable-train-data-part1/42487919/seismicCubes_RFC_fullstack_2024.42487919.npy'

## Valid

In [3]:
X_valid_path = '/home/simlab120/Denoise_comp/Pragyant/image-impeccable-train-data-part1/42502775/seismic_w_noise_vol_42502775.npy'
y_valid_path = '/home/simlab120/Denoise_comp/Pragyant/image-impeccable-train-data-part1/42502775/seismicCubes_RFC_fullstack_2024.42502775.npy'

In [4]:
shape = (1259,300,300)
# load the data
# make the data in 2d section
# make the train set and test set
# print the image also

In [5]:
import os
import pandas as pd
import numpy as np
from torchvision.io import read_image
from torch.utils.data import Dataset,DataLoader
import torch
# 2 files
# 1259*300*300 ---> 1259*300 in 300 images
# x,y load
# x,y sub sample
# pair
# return
class volDataset(Dataset):
    def __init__(self, x_slices, y_slices, transform=None):
        """
        x_slices: List of noisy seismic slices (each of shape 1259x300)
        y_slices: List of clean seismic slices (each of shape 1259x300)
        """
        self.x_slices = x_slices
        self.y_slices = y_slices
        self.transform = transform

    def __len__(self):
        return len(self.x_slices)

    def __getitem__(self, idx):
        """Returns one slice (1, 1259, 300)"""
        x_slice = torch.tensor(self.x_slices[idx], dtype=torch.float32).unsqueeze(0)  # (1, 1259, 300)
        y_slice = torch.tensor(self.y_slices[idx], dtype=torch.float32).unsqueeze(0)  # (1, 1259, 300)

        if self.transform:
            x_slice = self.transform(x_slice)
            y_slice = self.transform(y_slice)

        return x_slice, y_slice

In [6]:
from sklearn.model_selection import train_test_split
# Convert volume to 2D slices
# Load seismic volumes
X_volume = np.load('/home/simlab120/Denoise_comp/Pragyant/image-impeccable-train-data-part1/42487393/seismic_w_noise_vol_42487393.npy',allow_pickle=True)  # (1259, 300, 300)
y_volume = np.load('/home/simlab120/Denoise_comp/Pragyant/image-impeccable-train-data-part1/42487393/seismicCubes_RFC_fullstack_2024.42487393.npy',allow_pickle=True)  # (1259, 300, 300)

X_volume_t = np.load(X_test_path,allow_pickle=True)
y_volume_t = np.load(y_test_path,allow_pickle=True)

X_volume_v = np.load(X_valid_path,allow_pickle=True)
y_volume_v = np.load(y_valid_path,allow_pickle=True)
# Convert volume to 2D slices

X_slices = [X_volume[:, :, i] for i in range(X_volume.shape[2])]  # List of 300 slices
y_slices = [y_volume[:, :, i] for i in range(y_volume.shape[2])]  # List of 300 slices

X_slices_t = [X_volume_t[:, :, i] for i in range(X_volume_t.shape[2])]
y_slices_t = [y_volume_t[:, :, i] for i in range(y_volume_t.shape[2])]

X_slices_v = [X_volume_v[:, :, i] for i in range(X_volume_v.shape[2])]
y_slices_v = [y_volume_v[:, :, i] for i in range(y_volume_v.shape[2])]

# Create datasets
train_dataset = volDataset(X_slices, y_slices)
valid_dataset = volDataset(X_slices_v,y_slices_v)

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(valid_dataset,batch_size=8, shuffle=False)

# Check batch shapes
for x_batch, y_batch in train_loader:
    print("Train Batch Shape:", x_batch.shape, y_batch.shape)  # Expected: (8, 1, 1259, 300)
    break
for x_batch, y_batch in val_loader:
    print("valid Batch Shape:", x_batch.shape, y_batch.shape)  # Expected: (8, 1, 1259, 300)
    break

Train Batch Shape: torch.Size([8, 1, 1259, 300]) torch.Size([8, 1, 1259, 300])
valid Batch Shape: torch.Size([8, 1, 1259, 300]) torch.Size([8, 1, 1259, 300])


# Model

In [7]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [8]:
class Crop2d(nn.Module):
    def __init__(self, left=10, right=10, top=21, bottom=0):
        super(Crop2d, self).__init__()
        self.left = left
        self.right = right
        self.top = top
        self.bottom = bottom

    def forward(self, x):
        return x[:, :, self.top:x.shape[2] - self.bottom, self.left:x.shape[3] - self.right]

In [9]:
import timm
from segmentation_models_pytorch.decoders.unetplusplus import UnetPlusPlus

/home/simlab120/anaconda3/envs/torch/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.models.vision_transformer import vit_h_14, ViT_H_14_Weights


class Crop2d(nn.Module):
    """Crops a fixed amount from each side of a 4D tensor (B, C, H, W)."""
    def __init__(self, left, right, top, bottom):
        super().__init__()
        self.left = left
        self.right = right
        self.top = top
        self.bottom = bottom

    def forward(self, x):
        return x[:, :, self.top:x.shape[2]-self.bottom, self.left:x.shape[3]-self.right]


class ViTUNet(nn.Module):
    """ViT-H/14 as encoder without UNet decoder, using learned positional embeddings."""
    def __init__(self, in_channels=1, num_classes=1):
        super().__init__()

        # Load pretrained ViT-H/14
        vit = vit_h_14(weights=ViT_H_14_Weights.IMAGENET1K_SWAG_E2E_V1)
        self.encoder = vit

        # Modify conv_proj to accept single-channel images.
        self.encoder.conv_proj = nn.Conv2d(in_channels, 1280, kernel_size=14, stride=14, padding=0)

        # Adjust zero padding and cropping (Left, Right, Top, Bottom)
        self.zero_pad = nn.ZeroPad2d((10, 10, 13, 0))
        self.crop = Crop2d(10, 10, 13, 0)

        # Final classifier head: Map 1280 channels to 1 (grayscale output)
        self.head = nn.Conv2d(1280, num_classes, kernel_size=1)

    def forward(self, x):
        B, _, orig_H, orig_W = x.shape

        # Zero-pad input
        x_padded = self.zero_pad(x)  # Shape: (B, C, orig_H+13, orig_W+20)

        # Conv projection: produces (B, 1280, H_p, W_p)
        features = self.encoder.conv_proj(x_padded)
        B, C, H_p, W_p = features.shape  # Expect C == 1280

        # Flatten spatial dimensions → (B, N, 1280)
        features = features.flatten(2).transpose(1, 2)

        # Prepend class token → (B, N+1, 1280)
        class_token = self.encoder.class_token.expand(B, -1, -1)
        tokens = torch.cat([class_token, features], dim=1)

        # Desired token count:
        new_seq_len = 1 + (H_p * W_p)

        # ---- Update the transformer encoder's positional embeddings ----
        if hasattr(self.encoder.encoder, 'pos_embedding'):
            orig_pos_embed = self.encoder.encoder.pos_embedding  # shape: (1, orig_seq_len, 1280)
        else:
            raise KeyError("Transformer encoder has no 'pos_embedding' attribute.")

        if orig_pos_embed.shape[1] != new_seq_len:
            class_pos_embed = orig_pos_embed[:, :1, :]  # (1,1,1280)
            patch_pos_embed = orig_pos_embed[:, 1:, :]    # (1, orig_N, 1280)
            orig_N = patch_pos_embed.shape[1]
            orig_grid_size = int(orig_N ** 0.5)  # assume square grid
            patch_pos_embed = patch_pos_embed.reshape(1, orig_grid_size, orig_grid_size, C).permute(0, 3, 1, 2)
            new_grid = (H_p, W_p)
            patch_pos_embed = F.interpolate(patch_pos_embed, size=new_grid, mode='bilinear', align_corners=False)
            patch_pos_embed = patch_pos_embed.permute(0, 2, 3, 1).reshape(1, -1, C)
            new_pos_embed = torch.cat([class_pos_embed, patch_pos_embed], dim=1)
        else:
            new_pos_embed = orig_pos_embed

        self.encoder.encoder.pos_embedding = nn.Parameter(new_pos_embed)

        # Add positional embedding to tokens.
        tokens = tokens + self.encoder.encoder.pos_embedding

        # Transformer encoding.
        tokens = self.encoder.encoder(tokens)  # (B, N+1, 1280)

        # Remove class token and reshape to (B, 1280, H_p, W_p)
        patch_tokens = tokens[:, 1:, :].transpose(1, 2).reshape(B, C, H_p, W_p)

        # Resize output back to padded dimensions.
        x_out = F.interpolate(patch_tokens, size=(x_padded.shape[2], x_padded.shape[3]),
                              mode="bilinear", align_corners=False)

        # Crop back to original dimensions.
        x_out = self.crop(x_out)

        # Final classifier head: map 1280 channels to 1
        x_out = self.head(x_out)

        return x_out


# Test the model
model = ViTUNet()
x = torch.randn(1, 1, 1259, 300)  # (B, Channels, H, W)
out = model(x)
print(out.shape)  # Expected output: (1, 1, 1259, 300)


torch.Size([1, 1, 1259, 300])


In [11]:
pip install torcheval

Note: you may need to restart the kernel to use updated packages.


In [12]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define model
model = ViTUNet()

# Define Loss Function (MSE for denoising)
criterion = nn.MSELoss()

# Define Optimizer (Adam works well)
optimizer = optim.Adam(model.parameters(), lr=0.1)

In [13]:
# Check batch shapes
for x_batch, y_batch in train_loader:
    print("Train Batch Shape:", x_batch.shape, y_batch.shape)  # Expected: (batch_size, 1, 1259, 300)
    break

Train Batch Shape: torch.Size([8, 1, 1259, 300]) torch.Size([8, 1, 1259, 300])


In [14]:
from torchmetrics import StructuralSimilarityIndexMeasure
ssim_metric = StructuralSimilarityIndexMeasure()

/home/simlab120/anaconda3/envs/torch/lib/python3.12/site-packages/torchmetrics/utilities/prints.py:62: FutureWarning: Importing `StructuralSimilarityIndexMeasure` from `torchmetrics` was deprecated and will be removed in 2.0. Import `StructuralSimilarityIndexMeasure` from `torchmetrics.image` instead.
  _future_warning(


In [ ]:
import matplotlib.pyplot as plt

In [16]:
# Number of epochs
num_epochs = 200

# Training loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    running_ssim = 0.0

    for x_batch, y_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(x_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        batch_ssim = ssim_metric(outputs.detach(), y_batch.detach())
        running_ssim += batch_ssim.item()

    # Validation phase
    model.eval()
    val_loss = 0.0
    val_ssim = 0.0
    with torch.no_grad():
        for x_val, y_val in val_loader:
            val_outputs = model(x_val)
            val_loss += criterion(val_outputs, y_val).item()
            val_ssim += ssim_metric(val_outputs, y_val).item()

    # Prediction on a single slice from X_slices_t
    random_slice = X_slices_t[torch.randint(0, len(X_slices_t), (1,)).item()]

    # Convert to PyTorch tensor and add batch and channel dimensions
    random_slice = torch.tensor(random_slice, dtype=torch.float32).unsqueeze(0).unsqueeze(0)

    prediction = model(random_slice).detach()

    # Visualize the original and predicted slice
    plt.subplot(1, 2, 1)
    plt.title("Original Slice")
    plt.imshow(random_slice.squeeze().cpu().numpy(), cmap='seismic')

    plt.subplot(1, 2, 2)
    plt.title("Predicted Slice")
    plt.imshow(prediction.squeeze().cpu().numpy(), cmap='seismic')
    plt.show()

    print(f"Epoch [{epoch + 1}/{num_epochs}], Train Loss: {running_loss / len(train_loader):.4f}, Train SSIM: {running_ssim / len(train_loader):.4f}, \
          Val Loss: {val_loss / len(val_loader):.4f}, Val SSIM: {val_ssim / len(val_loader):.4f}")


print("Training complete!")


RuntimeError: [enforce fail at alloc_cpu.cpp:117] err == 0. DefaultCPUAllocator: can't allocate memory: you tried to allocate 16672358400 bytes. Error code 12 (Cannot allocate memory)

: 